# ME317 Computational Fluid Dynamics 

## Assignment 2 - Drishika Nadella (181ME222)

### This assignment has been solved using the Anaconda Jupyter Notebook Environment in Python3.

#### Question 1

The upward velocity of a rocket is given at three different times in Table 1.

| Time t (s)      | Velocity v (m/s) |
| ----------------| ---------------- |
| 5               | 106.8            |
| 8               | 177.2            |
| 12              | 279.2            |

The velocity data is approximated as a polynomial as:

$$ v(t) = a_1t^2 + a_2t + a_3 $$

where $ 5 \leq t \leq 12 $.

The coefficients $a_1$, $a_2$ and $a_3$ for the above given expression are given as:

$\begin{bmatrix}25 & 5 & 1 \\ 64 & 8 & 1 \\ 144 & 12 & 1 \end{bmatrix}$$\begin{bmatrix}a_1 \\ a_2 \\ a_3 \end{bmatrix}$ = $\begin{bmatrix} 106.8 \\ 177.2 \\ 279.2 \end{bmatrix}$

Find the velocity at t = 6 seconds.

In [1]:
# Importing the necessary libraries
import numpy as np
import sys
import timeit

Let us first solve this problem analytically, so we know what answers to expect.

In [2]:
# Initializing the variables

# Initializing the coefficient matrix
A = np.array([[25, 5, 1], [64, 8, 1], [144, 12, 1]])

#Initializing the RHS matrix
B = np.array([106.8, 177.2, 279.2])

In [3]:
# Solving the system of linear equations analytically

final = np.linalg.inv(A).dot(B)
print(final)

[ 0.29047619 19.69047619  1.08571429]


Therefore, the expected solutions are $a_1 = 0.29047619$, $a_2 = 19.69047619$ and $a_3 = 1.08571429$.

Now, solving the system of equations numerically:

Let us try to solve this problem using the Gauss-Seidel method.

### Gauss-Seidel Method

According to the Gauss Seidel method, the system of equations will be as follows:

$$ a_1^{k+1} = \frac{106.8 - 5a_2^k - a_3^k}{25} $$
$$ a_2^{k+1} = \frac{177.2 - 64a_1^{k+1} - a_3^k}{8} $$
$$ a_3^{k+1} = \frac{279.2 - 144a_1^{k+1} - 12a_2^{k+1}}{1} $$

We use the most recent iteration of the three variables while finding the values for the other variables.

In [4]:
# Initializing the variables

nt = 50                                        # Maximum number of iterations possible
error = 1E-6                                   # Maximum permissible error
c = [[25, 5, 1], [64, 8, 1], [144, 12, 1]]     # Coefficient matrix
d = [106.8, 177.2, 279.2]                      # RHS Matrix
a = [1, 2, 5]                                  # Initial guesses for the answers a1, a2 and a3

In [5]:
def GaussSeidel(nt, error, a, c, d):
    
    """
    This function solves the system of linear equations using the Gauss Seidel method as explained above.
    Here, nt = maximum number of iterations
    error = maximum permissible error of the answers to the true values
    a = matrix for the initial guesses of a1, a2, a3
    c = coefficient matrix
    d = RHS matrix
    The function prints out the a1, a2, a3 values every 10 iterations.
    """
    
    for n in range(nt+1):
        
        # x, y, z store the old values of a1, a2, a3
        x = a[0]
        y = a[1]
        z = a[2]
        
        # Performing Gauss Seidel iteration for all three variables
        # Note how a2 calculation uses the most recent a1, and a3 calculation uses the most recent a1 and a2.
        a[0] = (d[0] - c[0][1]*y - c[0][2]*z)/c[0][0]
        a[1] = (d[1] - c[1][0]*a[0] - c[1][2]*z)/c[1][1]
        a[2] = (d[2] - c[2][0]*a[0] - c[2][1]*a[1])/c[2][2]
        
        # Checking for convergence
        if max(abs(x-a[0]), abs(y-a[1]), abs(z-a[2])) < error:
            print("\n")
            print("Finally, a1 = %f, a2 = %f, a3 = %f"% (a[0], a[1], a[2]))
            print("It took %d iterations"% (n+1))
            break
        
        # Prints out the values every 10 iterations
        elif (n+1)%10==0:
            print("Iteration number: ", n+1)
            print("a1 = %f \n a2 = %f \n a3 = %f \n"% (a[0], a[1], a[2]))
            print("\n")

In [6]:
# Calling the Gauss Seidel function
GaussSeidel(nt, error, a, c, d)

# Printing the expected analytical result
print("Expected result: a1 = %f \n a2 = %f \n a3 = %f"%(final[0], final[1], final[2]))

Iteration number:  10
a1 = 988378.793288 
 a2 = -5671869.896305 
 a3 = -74263828.277822 



Iteration number:  20
a1 = 1509210001999.948975 
 a2 = -8660719631421.269531 
 a3 = -113397604710658.234375 



Iteration number:  30
a1 = 2304496976556495104.000000 
 a2 = -13224536134145269760.000000 
 a3 = -173153131014392086528.000000 



Iteration number:  40
a1 = 3518865040598500695343104.000000 
 a2 = -20193282245095887279751168.000000 
 a3 = -264397178905033435592523776.000000 



Iteration number:  50
a1 = 5373151407839446864779418796032.000000 
 a2 = -30834249586816195591478917988352.000000 
 a3 = -403722807687086019444887800250368.000000 



Expected result: a1 = 0.290476 
 a2 = 19.690476 
 a3 = 1.085714


We can see that even after 50 iterations, the values diverge drastically from the expected result and the errors compound. This is because Gauss-Seidel solver is an iterative solver. Iterative solvers require the "diagonal dominance" condition to be satisfied. The diagonal dominance condition is:

- The coefficient of the diagonal of the matrix must be at least equal to the sum of the other coefficients in that row
- At least one row with a diagonal coefficient greater than the sum of the other coefficients in that row

Let us test the diagonal dominance condition on the rows of the coefficient matrix.

- Row 1

$ 25 \geq 5 + 1 $ condition is satisfied.

- Row 2

$ 8 \geq 64 + 1 $ condition is not satisfied.

- Row 3

$ 1 \geq 144 + 12 $ condition is not satisfied.

Therefore, rows 2 and 3 fail the first condition in the diagonal dominance requirement. Therefore, iterative solvers such as the Gauss-Seidel method are **not valid** for the problem. 

We need to use direct solvers to solve the problem.

### Gauss Elimination Method

The Gauss Elimination method is a type of direct solver that obtains the solutions to a system of linear equations. Since it is a direct solver, it does not require diagonal dominance.

The steps involved in the Gauss Elimination method are:

- Swapping two rows
- Multiplying a row by a nonzero number
- Adding a multiple of one row to another row

These operations are performed until the lower left-hand corner of the matrix is filled with zeros, as much as possible.

In [7]:
def Gauss(a, n):
    """
    This function solves a set of linear equations using the Gauss Elimination method.
    Here, a = augmented matrix that contains the coefficient and the RHS matrices
    n = number of unknowns
    The function returns the three unknowns a1, a2, a3
    """
    # Defining the array that will contain the solution
    x = np.zeros(n)
    
    # Gaussian Elimination method
    for i in range(n):
        for j in range(i+1, n):
            ratio = a[j][i]/a[i][i]

            for k in range(n+1):
                a[j][k] = a[j][k] - ratio * a[i][k]

    x[n-1] = a[n-1][n]/a[n-1][n-1]
    
    # Back substitution
    for i in range(n-2,-1,-1):
        x[i] = a[i][n]

        for j in range(i+1,n):
            x[i] = x[i] - a[i][j]*x[j]

        x[i] = x[i]/a[i][i]

    return x

In [8]:
init = np.array([[25, 5, 1, 106.8], [64, 8, 1, 177.2], [144, 12, 1, 279.2]])
results = Gauss(init, 3)
print("The final results are: a1 = %f, a2 = %f, a3 = %f"% (results[0], results[1], results[2]))

The final results are: a1 = 0.290476, a2 = 19.690476, a3 = 1.085714


We can see that the above results we obtained with the Gauss Elimination method gave us the same results as the analytical method. Now, we need to find the value of velocity v at time t = 6 seconds.

In [9]:
t = 6
v = results[0]*t**2 + results[1]*t + results[2]
print("The velocity v at time %d seconds is %f m/s"%(t,v))

The velocity v at time 6 seconds is 129.685714 m/s


### Computing CPU time to execute the code

The CPU time to execute the code can be done using the timeit module in Python. It requires me to put the code whose execution time needs to be computed in a string. The final result is printed below.

In [10]:
mysetup = "import numpy as np"

mycode = """
def Gauss(a, n):
    x = np.zeros(n)
    
    for i in range(n):
        for j in range(i+1, n):
            ratio = a[j][i]/a[i][i]

            for k in range(n+1):
                a[j][k] = a[j][k] - ratio * a[i][k]

    x[n-1] = a[n-1][n]/a[n-1][n-1]

    for i in range(n-2,-1,-1):
        x[i] = a[i][n]

        for j in range(i+1,n):
            x[i] = x[i] - a[i][j]*x[j]

        x[i] = x[i]/a[i][i]

    return x

init = np.array([[25, 5, 1, 106.8], [64, 8, 1, 177.2], [144, 12, 1, 279.2]])
results = Gauss(init, 3)
print(results)
"""

print("The time taken for the Gauss Elimination code to execute is: %f"% (timeit.timeit(setup = mysetup, stmt = mycode, number = 1)))

[ 0.29047619 19.69047619  1.08571429]
The time taken for the Gauss Elimination code to execute is: 0.000609


### Results and Discussion

We can see that the velocity at time t=6 seconds is 129.685714 m/s. 

Therefore, we can see that for the given question, the Gauss-Seidel code is invalid as it does not satisfy the diagonal dominance condition, but the Gauss Elimination code is **valid**, since it is a direct solver that does not require the diagonal dominance condition to be satisfied.

We checked for the validity of the code by comparing the coefficient values obtained by the Gauss Elimination method with the coefficient values obtained by the analytical method at the beginning. They both match, thereby confirming the validity of the code.

The Gauss Elimination algorithm took around 0.0006 seconds to execute. 